In [1]:
using Distributed

In [2]:
#addprocs(4);
#@everywhere using Pkg; @everywhere Pkg.activate("/Users/briandepasquale/Projects/inProgress/pulse_input_DDM.jl")

addprocs(44);
@everywhere using Pkg; @everywhere Pkg.activate("/usr/people/briandd/Projects/pulse_input_DDM.jl")

In [3]:
@everywhere using pulse_input_DDM

┌ Info: Recompiling stale cache file /usr/people/briandd/.julia/compiled/v1.0/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1190


## Compare against monte carlo simulation dt

In [ ]:
#check that decimate on \lambda0 isn't doing something horrible.

In [ ]:
num_trials = map(i->[i], 100:500:4100)
nsessions = ones(Int,length(num_trials))
num_cells = map(i->[i], nsessions)

output = map((x,y,z)-> pulse_input_DDM.generate_syn_data_fit_CI(x,y,z;dt=1e-4), 
    nsessions, num_cells, num_trials)
pz, py = map(collect, zip(output...));

┌ Warning: Linesearch failed, using alpha = 0.0 and exiting optimization.
│ The linesearch exited with message:
│ Linesearch failed to converge, reached maximum iterations 1000.
└ @ Optim /usr/people/briandd/.julia/packages/Optim/Agd3B/src/utilities/perform_linesearch.jl:47
┌ Warning: Linesearch failed, using alpha = 0.0 and exiting optimization.
│ The linesearch exited with message:
│ Linesearch failed to converge, reached maximum iterations 1000.
└ @ Optim /usr/people/briandd/.julia/packages/Optim/Agd3B/src/utilities/perform_linesearch.jl:47
┌ Warning: Linesearch failed, using alpha = 0.0 and exiting optimization.
│ The linesearch exited with message:
│ Linesearch failed to converge, reached maximum iterations 1000.
└ @ Optim /usr/people/briandd/.julia/packages/Optim/Agd3B/src/utilities/perform_linesearch.jl:47
┌ Warning: Linesearch failed, using alpha = 0.0 and exiting optimization.
│ The linesearch exited with message:
│ Linesearch failed to converge, reached maximum iterations 100

In [ ]:
using PyPlot
num_rows, num_cols = 1,5
fig, ax = subplots(num_rows, num_cols, figsize=(9,3))
param_str = ["B", "λ"]
fs,lw,ms = 22,2,6

for j in 1:2  

    ax[j][:plot](num_trials, pz[1]["generative"][pz[1]["name"] .== param_str[j]][1] * ones(length(num_trials)))
    ax[j][:set_xlabel]("num trials")
    ax[j][:set_title]("$(param_str[j])")

    for k = 1:length(num_trials)

        x = pz[k]["final"][pz[k]["name"] .== param_str[j]]
        yerr = cat(pz[k]["final"][pz[k]["name"] .== param_str[j]] - 
            pz[k]["CI_minus"][pz[k]["name"] .== param_str[j]],
            pz[k]["CI_plus"][pz[k]["name"] .== param_str[j]] - 
            pz[k]["final"][pz[k]["name"] .== param_str[j]], dims=2)

        ax[j][:errorbar](num_trials[k], x, yerr = yerr'*ones(1,1), fmt="o", 
            elinewidth=lw, color = "black", markersize=ms)

    end        
end

for j in 3:5  

    ax[j][:plot](num_trials, py[1]["generative"][1][1][j-2] * ones(length(num_trials)))
    ax[j][:set_xlabel]("num trials")

    for k = 1:length(num_trials)

        x = py[k]["final"][1][1][j-2]
        yerr = cat(py[k]["final"][1][1][j-2] - 
            py[k]["CI_minus"][1][1][j-2],
            py[k]["CI_plus"][1][1][j-2] - 
            py[k]["final"][1][1][j-2], dims=2)

        ax[j][:errorbar](num_trials[k], x, yerr = yerr'*ones(1,1), fmt="o", 
            elinewidth=lw, color = "black", markersize=ms)

    end        
end

tight_layout()

## Test noiseless model

In [ ]:
nsessions = [1,2,3,4,5]
output = map((x,y,z)-> pulse_input_DDM.generate_syn_data_fit_CI(x,y,z;dt=1e-2), 
    nsessions, 
    [1*ones(Int,n) for n in 1:length(nsessions)], 
    [100*ones(Int,n) for n in 1:length(nsessions)])
pz, py = map(collect, zip(output...));

In [ ]:
using PyPlot
num_rows, num_cols = 1,2
fig, ax = subplots(num_rows, num_cols, figsize=(9,3))
param_str = ["B", "λ"]
fs,lw,ms = 22,2,6

for i in 1:1
    for j in 1:num_cols  
        
        ax[j][:plot](0:6, pz[1]["generative"][pz[1]["name"] .== param_str[j]][1] * ones(7))
        ax[j][:set_xlim](0,6)
        ax[j][:set_xlabel]("num sessions")
        ax[j][:set_title]("$(param_str[j])")
        
        for k = 1:length(pz)

            x = pz[k]["final"][pz[k]["name"] .== param_str[j]]
            yerr = cat(pz[k]["final"][pz[k]["name"] .== param_str[j]] - 
                pz[k]["CI_minus"][pz[k]["name"] .== param_str[j]],
                pz[k]["CI_plus"][pz[k]["name"] .== param_str[j]] - 
                pz[k]["final"][pz[k]["name"] .== param_str[j]], dims=2)
            #yerr[yerr .>= 1e4] .= 1e4

            ax[j][:errorbar](k, x, yerr = yerr'*ones(1,1), fmt="o", 
                elinewidth=lw, color = "black", markersize=ms)
            
        end        
    end
end

tight_layout()

In [ ]:
plot(pulse_input_DDM.f_py!([0.,10,80], collect(-8:0.1:8), 
        zeros(length(collect(-8:0.1:8))), f_str="softplus"))

## test stitching

In [ ]:
nsessions = [1,2,3,4,5,6,7,8]
output = map((x,y,z)-> pulse_input_DDM.generate_syn_data_fit_CI(x,y,z;dt=1e-3,n=53), 
    nsessions, 
    [1*ones(Int,n) for n in 1:length(nsessions)], 
    [100*ones(Int,n) for n in 1:length(nsessions)])
pz, py = map(collect, zip(output...));

In [ ]:
using PyPlot
num_rows, num_cols = 1,3
fig, ax = subplots(num_rows, num_cols, figsize=(9,3))
param_str = ["B", "λ", "σ_a"]
fs,lw,ms = 22,2,6

for i in 1:1
    for j in 1:num_cols  
        
        ax[j][:plot](0:9, pz[1]["generative"][pz[1]["name"] .== param_str[j]][1] * ones(10))
        ax[j][:set_xlim](0,9)
        ax[j][:set_xlabel]("num sessions")
        ax[j][:set_title]("$(param_str[j])")
        
        for k = 1:length(pz)

            x = pz[k]["final"][pz[k]["name"] .== param_str[j]]
            yerr = cat(pz[k]["final"][pz[k]["name"] .== param_str[j]] - 
                pz[k]["CI_minus"][pz[k]["name"] .== param_str[j]],
                pz[k]["CI_plus"][pz[k]["name"] .== param_str[j]] - 
                pz[k]["final"][pz[k]["name"] .== param_str[j]], dims=2)
            #yerr[yerr .>= 1e4] .= 1e4

            ax[j][:errorbar](k, x, yerr = yerr'*ones(1,1), fmt="o", 
                elinewidth=lw, color = "black", markersize=ms)
            
        end        
    end
end

tight_layout()
path="/usr/people/briandd/Projects/pulse_input_DDM.jl"
PyPlot.savefig(path*"/data/results/working/stitching.png")

In [ ]:
using PyPlot
num_rows, num_cols = 1,4
fig, ax = subplots(num_rows, num_cols, figsize=(9,3))
param_str = ["B", "λ", "σ_a","σ_s"]
fs,lw,ms = 22,2,6

for i in 1:1
    for j in 1:num_cols  
        
        ax[j][:plot](0:21, pz[1]["generative"][pz[1]["name"] .== param_str[j]][1] * ones(22))
        ax[j][:set_xlim](0,21)
        ax[j][:set_xlabel]("num sessions")
        ax[j][:set_title]("$(param_str[j])")
        
        for k = 1:length(pz)

            x = pz[k]["final"][pz[k]["name"] .== param_str[j]]
            yerr = cat(pz[k]["final"][pz[k]["name"] .== param_str[j]] - 
                pz[k]["CI_minus"][pz[k]["name"] .== param_str[j]],
                pz[k]["CI_plus"][pz[k]["name"] .== param_str[j]] - 
                pz[k]["final"][pz[k]["name"] .== param_str[j]], dims=2)
            #yerr[yerr .>= 1e4] .= 1e4

            ax[j][:errorbar](k, x, yerr = yerr'*ones(1,1), fmt="o", 
                elinewidth=lw, color = "black", markersize=ms)
            
        end        
    end
end

tight_layout()
path="/usr/people/briandd/Projects/pulse_input_DDM.jl"
PyPlot.savefig(path*"/data/results/working/stitching.png")